In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score,accuracy_score
import pickle

In [2]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\16409\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
df = pd.read_csv('train_data.csv',sep = ',', names =['Comments','Reviews'])
#dataset = df[1:].sample(n=10000, random_state = 1)
dataset =df[1:]

In [26]:
dataset

,Comments,Reviews
1,"This film is absolutely awful, but nevertheles...",0
2,Well since seeing part's 1 through 3 I can hon...,0
3,I got to see this film at a preview and was da...,1
4,This adaptation positively butchers a classic ...,0
5,Råzone is an awful movie! It is so simple. It ...,0
...,...,...
24996,With this movie being the only Dirty Harry mov...,1
24997,Any screen adaptation of a John Grisham story ...,1
24998,This film captured my heart from the very begi...,1
24999,A deplorable social condition triggers off the...,1


In [32]:
sum(dataset.Reviews)/25000

0.5

In [5]:
stopset = set(stopwords.words('english'))

In [16]:
vectorizer = TfidfVectorizer(use_idf = True,lowercase = True, strip_accents='ascii',stop_words=stopset)

In [27]:
X = vectorizer.fit_transform(dataset.Comments)
y = dataset.Reviews
pickle.dump(vectorizer, open('tranform.pkl', 'wb'))

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [34]:
from sklearn import tree
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import ensemble
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import metrics

classifiers = {
    'DecisionTreeClassifier': tree.DecisionTreeClassifier(),
    'LogisticRegression': linear_model.LogisticRegressionCV(),
    #Navies Bayes
    'MultinomialNB': naive_bayes.MultinomialNB(),
    'BernoulliNB': naive_bayes.BernoulliNB(),
    #'GaussianNB': naive_bayes.GaussianNB(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': ensemble.AdaBoostClassifier(),
    "XGBClassifier": XGBClassifier(random_state=0),
}

df_models = pd.DataFrame(columns = ['model', 
                                    #'run_time', 
                                    'avg_accy',
                                    'avg_accy_std',  
                                    'avg_recall',
                                    'avg_recall_std',
                                    'avg_precision',
                                    'avg_precision_std',
                                    'avg_f1',
                                    'avg_f1_std',
                                    #'avg_matthew_corcoef',
                                    #'avg_matthew_corcoef_std',
                                    'avg_roc_auc',
                                    'avg_roc_auc_std', 
                                     ])
df_test_accy = pd.DataFrame(columns = ['model','accy','recall','precision','auc'])

scorer = {'accuracy_score':metrics.make_scorer(metrics.accuracy_score),
          'f1_score':metrics.make_scorer(metrics.f1_score),
          'recall_score':metrics.make_scorer(metrics.recall_score),
          'precision_score':metrics.make_scorer(metrics.average_precision_score),
          #'matthew_corrcoef':metrics.make_scorer(metrics.matthews_corrcoef),
          'roc_auc_score':metrics.make_scorer(metrics.roc_auc_score) 
         }

for key in classifiers:
    print('*', key)
    #start_time = time.time()
    classifier = classifiers[key]
    model = classifier.fit(X_train,y_train)
    #cvs = model_selection.RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
    #cv_scores = model_selection.cross_validate(classifier, X_train, y_train, cv=cvs, scoring = scorer)
    y_pred = model.predict(X_test)
    pred_score = accuracy_score(y_test,y_pred)*100
    recall = metrics.recall_score(y_test,y_pred)*100
    precision = metrics.precision_score(y_test,y_pred)*100
    auc = metrics.roc_auc_score(y_test,y_pred)*100
    #print("Model: ",key,"\ntest accuracy score: ",pred_score)
    df_test_accy = df_test_accy.append({'model':key, 'accy':pred_score,'recall':recall,'precision':precision,'auc':auc}, ignore_index = True)

    
    #df_models = df_models.append(row, ignore_index=True)
    

* DecisionTreeClassifier


C:\Users\16409\AppData\Local\Temp\ipykernel_23476\2218997216.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_accy = df_test_accy.append({'model':key, 'accy':pred_score,'recall':recall,'precision':precision,'auc':auc}, ignore_index = True)


* LogisticRegression


C:\Users\16409\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\16409\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stab

* MultinomialNB
* BernoulliNB
* RandomForestClassifier


C:\Users\16409\AppData\Local\Temp\ipykernel_23476\2218997216.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_accy = df_test_accy.append({'model':key, 'accy':pred_score,'recall':recall,'precision':precision,'auc':auc}, ignore_index = True)


* AdaBoostClassifier


C:\Users\16409\AppData\Local\Temp\ipykernel_23476\2218997216.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_accy = df_test_accy.append({'model':key, 'accy':pred_score,'recall':recall,'precision':precision,'auc':auc}, ignore_index = True)


* XGBClassifier


C:\Users\16409\AppData\Local\Temp\ipykernel_23476\2218997216.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_accy = df_test_accy.append({'model':key, 'accy':pred_score,'recall':recall,'precision':precision,'auc':auc}, ignore_index = True)


In [35]:
df_test_accy

,model,accy,recall,precision,auc
0,DecisionTreeClassifier,71.60,71.226224,71.283921,71.595714
1,LogisticRegression,89.58,91.622825,87.820016,89.603425
2,MultinomialNB,87.28,87.252125,87.040775,87.279680
3,BernoulliNB,85.06,81.060299,87.773883,85.014136
4,RandomForestClassifier,85.42,85.997572,84.728868,85.426623
5,AdaBoostClassifier,79.76,84.904897,76.653270,79.818996
6,XGBClassifier,84.36,87.373533,82.122480,84.394556


In [38]:
clf = linear_model.LogisticRegressionCV()
clf.fit(X,y)

C:\Users\16409\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\16409\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stab

LogisticRegressionCV()

In [40]:
accuracy_score(y_test,clf.predict(X_test))*100

96.6

In [41]:
filename = 'nlp_model.pkl'
pickle.dump(clf, open(filename, 'wb'))